### Using openai package

In [14]:
from openai import OpenAI
client = OpenAI(
    base_url="http://192.168.50.155:8000/v1/",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="google/gemma-2-2b-it",
  seed=1337,
  temperature=0.0,
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! 👋  \n\nHow can I help you today? 😊 \n', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[])


In [15]:
completion = client.chat.completions.create(
  model="google/gemma-2-2b-it",
  messages=[
    {"role": "user", "content": "Classify this sentiment: vLLM is wonderful!"}
  ],
  extra_body={
    "guided_choice": ["positive", "negative"]
  }
)

print(completion.choices[0].message)

ChatCompletionMessage(content='positive', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[])


In [16]:
completion = client.chat.completions.create(
  model="google/gemma-2-2b-it",
  messages=[
    {"role": "user", "content": "Hvem har været statsminister i danmark?"}
  ],
  extra_body={
    "guided_choice": ["Frederiksen", "Biden"]
  }
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Frederiksen', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[])


In [ ]:
!python main.py --models google/gemma-2-2b-it --datasets Lydmaskinen_qa_easy --replications 1 --test_modes constrained --temperatures 0.0 --max_task_tokens 100 --max_response_tokens 50 --refine False --seed True --suffix Experiment_name --api_url http://192.168.50.155:8000/v1/completions

## vLLM specific:
https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html

In [3]:
url = "http://192.168.50.155:8000/v1"

In [4]:
import os
from openai import AsyncOpenAI
from outlines import models
from outlines.models.openai import OpenAIConfig


client = AsyncOpenAI(
    api_key="None",#os.environ.get("PROVIDER_KEY"),
    #base_url="http://192.168.50.155:8000/v1/completions"
    base_url=url
)
config = OpenAIConfig("google/gemma-2-2b-it")
model = models.openai(client, config)

In [5]:

from outlines import models, generate
generator = generate.choice(model, ["skirt", "dress", "pen", "jacket"])
answer = generator("Pick the odd word out: skirt, dress, pen, jacket")
print(answer)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
import os
from openai import AsyncOpenAI
from outlines import models, generate
from outlines.models.openai import OpenAIConfig

client = AsyncOpenAI(
    api_key="none",  # Replace with a valid API key if required
    base_url="http://192.168.50.155:8000/v1/completions"
)

config = OpenAIConfig("google/gemma-2-2b-it")
model = models.openai(client, config)

generator = generate.choice(model, ["skirt", "dress", "pen", "jacket"])

async def get_answer():
    answer = await generator("Pick the odd word out: skirt, dress, pen, jacket")
    print(answer)

import asyncio
asyncio.run(get_answer())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Logging

In [31]:
import logging

class DebugOpenAI(models.OpenAI):
    def __call__(self, prompt, max_tokens=None, stop_at=None, system_prompt=None, temperature=None, samples=None):
        response = super().__call__(prompt, max_tokens, stop_at, system_prompt, temperature, samples)
        logging.debug(f"Raw API response: {response}")
        return response

model = DebugOpenAI(
    "google/gemma-2-2b-it",
    base_url="http://192.168.50.155:8000/v1/",
    api_key="hunter12",
)

TypeError: OpenAI.__init__() got an unexpected keyword argument 'base_url'

In [38]:
import ollama

model = ollama.Client()
response = model.chat(model='gemma2:2b', messages=[
    {
        'role': 'user',
        'content': 'hi there',
    },
])
print(response['message']['content'])

Hello! 👋 How can I help you today? 😄 



In [41]:
from outlines import models, generate

# Ollama
model = models.openai(
    "gemma2:2b",  # This should be the first, positional argument
    #base_url="http://localhost:11434/api/chat",
    #base_url="http://localhost:11434",
    base_url="http://localhost:11434/api/generate",
    api_key="hunter12"
)

generator = generate.text(model)
print(generator("hi there"))

NotFoundError: 404 page not found

In [28]:
import pkg_resources

def get_version(package):
    try:
        return pkg_resources.get_distribution(package).version
    except pkg_resources.DistributionNotFound:
        return "Package not found"

print(get_version("outlines"))
print(get_version("json"))

0.1.1
Package not found


In [30]:
import json
print(json.__version__)

2.0.9


In [26]:
import outlines.models as models
from outlines import generate


model = models.openai(
    "google/gemma-2-2b-it", #"gemma2:2b"
    base_url="http://192.168.50.155:8000/v1/", #"https://openrouter.ai/api/v1", #http://localhost:11434/api/generate
    api_key="hunter12",
)

generator = generate.text(model)
print(generator("hi there"))

Hi there! 😄 

What can I help you with today? 😊 



In [27]:
generator = generate.choice(model, ["Enron", "FTX", "Silicon Valley Bank"])
print(generator("What will be the biggest company in the world in 2025?"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
import outlines.models as models
from outlines import generate
from pydantic import BaseModel, ConfigDict


#model = models.openai("gpt-4o-mini")

# smoke test json schema model creation
model_for_json_schema = model.new_with_replacements(response_format={"type": "json_schema"})
print(model.config)
# OpenAIConfig(model='gpt-4o-mini', frequency_penalty=0, logit_bias={}, max_tokens=None, n=1, presence_penalty=0, response_format=None, seed=None, stop=None, temperature=1.0, top_p=1, user='')
print(model_for_json_schema.config)
# OpenAIConfig(model='gpt-4o-mini', frequency_penalty=0, logit_bias={}, max_tokens=None, n=1, presence_penalty=0, response_format={'type': 'json_schema'}, seed=None, stop=None, temperature=1.0, top_p=1, user='')

generator = generate.choice(model, ["Enron", "FTX", "Silicon Valley Bank"])
print(generator("What will be the biggest company in the world in 2025?"))

OpenAIConfig(model='google/gemma-2-2b-it', frequency_penalty=0, logit_bias={}, max_tokens=None, n=1, presence_penalty=0, response_format=None, seed=None, stop=None, temperature=1.0, top_p=1, user='')
OpenAIConfig(model='google/gemma-2-2b-it', frequency_penalty=0, logit_bias={}, max_tokens=None, n=1, presence_penalty=0, response_format={'type': 'json_schema'}, seed=None, stop=None, temperature=1.0, top_p=1, user='')


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
generator = generate.choice(model, ["Enron", "FTX", "Silicon Valley Bank"])
print(generator("What will be the biggest company in the world in 2025?"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)